**Install lib**

In [ ]:
!pip install tsfresh


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install -U --no-cache-dir gdown --pre

In [ ]:
!gdown https://drive.google.com/a/gm.uit.edu.vn/uc?id=1pXJZ9eHp6DWkQ5MhCzmWYsKyLQEDiodz&export=download
!tar xzf /content/vn_sbert_deploy.tar.gz

In [ ]:
!pip -q install transformers
!pip -q install vncorenlp
!pip -q install fairseq
!pip -q install fastBPE

!pip -q install fastapi
!pip -q install uvicorn
!pip -q install pyngrok
!pip -q install pyyaml==5.4.1 


In [ ]:
!pip install category_encoders

In [ ]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

In [ ]:
!git clone https://github.com/DatCanCode/sentence-transformers
!cd sentence-transformers; pip install -e .

**Import lib**

In [ ]:
import numpy as np
import pandas as pd
import tsfresh
from sklearn.decomposition import PCA
from sklearn.feature_extraction import DictVectorizer
import category_encoders as ce


In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
train_work = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_edited.csv')
test_work = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_edited.csv')

In [ ]:
train_work = train_work.sort_values(['id_bh', 'from_date'])
test_work = test_work.sort_values(['id_bh', 'from_date'])

train_work = train_work.reset_index(drop=True)
test_work = test_work.reset_index(drop=True)

In [ ]:
train_work = train_work.fillna(-1)
test_work = test_work.fillna(-1)

In [ ]:
time_train = (train_work.groupby('id_bh').last()['to_date'] - train_work.groupby('id_bh').first()['from_date']).values
time_test = (test_work.groupby('id_bh').last()['to_date'] - test_work.groupby('id_bh').first()['from_date']).values

In [ ]:
train_work 

,bithYear,gender,id_bh,id,id_management,id_office,company_type,job/role,from_date,to_date,employee_lv,label,latitude,longitude
0,1963,0,100000725,4686,100,HW001Z,-1,bo doi,19830300,19870200,-1,5,16.16667,107.83333
1,1963,0,100000725,4686,100,HW001Z,-1,hoc,19870300,19900500,-1,5,16.16667,107.83333
2,1963,0,100000725,4686,100,HW001Z,-1,cong tac,19900600,19920800,-1,5,16.16667,107.83333
3,1963,0,100000725,4686,100,HW001Z,-1,can su,19920900,19930300,0,5,16.16667,107.83333
4,1963,0,100000725,4686,100,HW001Z,-1,can su,19930400,19960800,4,5,16.16667,107.83333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235386,1995,0,9820174207,17155,1900,TW0018Z,-1,cong nhan cat got kim loai,20160600,20161200,6,2,21.59422,105.84817
235387,1995,0,9820174207,17155,1900,TW0018Z,-1,cong nhan cat got kim loai,20170100,20170100,7,2,21.59422,105.84817
235388,1995,0,9820174207,17155,1900,HZ0000Z,6,chien si nghia vu,20170200,20190600,2,2,16.16667,107.83333
235389,1995,0,9820174207,17155,1900,HZ0000Z,6,chien si nghia vu,20190700,20200100,2,2,16.16667,107.83333


**Time series**

In [ ]:
train_series = train_work[['id_bh','from_date','employee_lv']]
test_series = test_work[['id_bh','from_date','employee_lv']]   
  
df_features = tsfresh.extract_features(train_series, column_id='id_bh', column_sort='from_date', default_fc_parameters=tsfresh.feature_extraction.MinimalFCParameters())
df_features_test = tsfresh.extract_features(test_series, column_id='id_bh', column_sort='from_date', default_fc_parameters=tsfresh.feature_extraction.MinimalFCParameters())

df_features['id_bh'] = df_features.index
df_features_test['id_bh'] = df_features_test.index

df_features.columns

Feature Extraction: 100%|██████████| 18134/18134 [00:14<00:00, 1294.02it/s]


Index(['employee_lv__sum_values', 'employee_lv__median', 'employee_lv__mean',
       'employee_lv__length', 'employee_lv__standard_deviation',
       'employee_lv__variance', 'employee_lv__root_mean_square',
       'employee_lv__maximum', 'employee_lv__absolute_maximum',
       'employee_lv__minimum', 'id_bh'],
      dtype='object')

**Fill N/A**

In [ ]:
train_work = train_work[train_work['id_bh'] != train_work['id_bh'].shift(-1)]
test_work = test_work[test_work['id_bh'] != test_work['id_bh'].shift(-1)]

In [ ]:
train_work['time'] = time_train
test_work['time'] = time_test

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
train_work = train_work.join(df_features.set_index('id_bh'), on='id_bh')
test_work = test_work.join(df_features_test.set_index('id_bh'), on='id_bh')

**Transform job/role**

In [ ]:
all = train_work.append(test_work, ignore_index=True)
all

,bithYear,gender,id_bh,id,id_management,id_office,company_type,job/role,from_date,to_date,...,employee_lv__sum_values,employee_lv__median,employee_lv__mean,employee_lv__length,employee_lv__standard_deviation,employee_lv__variance,employee_lv__root_mean_square,employee_lv__maximum,employee_lv__absolute_maximum,employee_lv__minimum
0,1963,0,100000725,4686,100,HW01180,6,chuyen vien chinh tp,20210600,20220400,...,148.0,8.0,8.222222,18.0,6.187403,38.283951,10.290233,17.0,17.0,-1.0
1,1971,0,100007067,29905,2400,TA0002A,1,nhan vien ban xang dau,20190100,20220400,...,63.0,6.0,6.300000,10.0,1.615549,2.610000,6.503845,9.0,9.0,4.0
2,1970,0,100007555,4503,109,TI3361I,-1,nhan vien,20211000,20220400,...,102.0,6.0,6.000000,17.0,3.067860,9.411765,6.738825,11.0,11.0,-1.0
3,1970,0,100008102,2939,100,HW0013Z,-1,chuyen vien chinh uy vien thuong truc uy ban t...,20200800,20220400,...,235.0,13.5,11.750000,20.0,3.766630,14.187500,12.338963,18.0,18.0,5.0
4,1965,0,100008777,8967,114,HN05360,-1,pho chu tich uy ban nhan dan huyen,20201000,20220400,...,204.0,10.0,10.200000,20.0,4.154516,17.260000,11.013628,17.0,17.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45631,1984,0,9714758429,6662,101,TA7401A,1,nhan vien ki thuat nha tram chi nhanh ki thuat...,20200100,20220400,...,45.0,9.0,6.428571,7.0,4.169863,17.387755,7.662525,11.0,11.0,1.0
45632,1987,0,9714758435,745,101,TA7401A,1,nhan vien ki thuat day may chi nhanh ki thuat ...,20210200,20210700,...,56.0,9.5,7.000000,8.0,4.183300,17.500000,8.154753,11.0,11.0,1.0
45633,1994,0,9715056070,49545,109,TI6860I,1,giam doc,20190400,20220400,...,24.0,5.0,6.000000,4.0,2.345208,5.500000,6.442049,10.0,10.0,4.0
45634,1989,0,9715637756,8811,105,QZ04360,1,chuyen gia thi truong,20220300,20220400,...,49.0,10.0,12.250000,4.0,6.759253,45.687500,13.991069,23.0,23.0,6.0


In [ ]:
encoder = ce.BinaryEncoder(cols=['id_office'],return_df=True)
all = encoder.fit_transform(all)

In [ ]:
all['label']

0        5.0
1        4.0
2        2.0
3        5.0
4        6.0
        ... 
45631    NaN
45632    NaN
45633    NaN
45634    NaN
45635    NaN
Name: label, Length: 45636, dtype: float64

In [ ]:
X_train = all[:27502]
X_test = all[27502:]

In [ ]:
%cd /content/sentence-transformers/
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/content/vn_sbert_deploy/phobert_base_mean_tokens_NLI_STS')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = []
for index, row in all.iterrows():
    corpus.append(str(row['job/role']))
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
result = X.toarray()

In [ ]:
Y_train = X_train['label']
X_train = X_train.drop(['label', 'id_bh', 'id', 'job/role', 'from_date', 'to_date'], axis = 1)

X_test_id = X_test['id_bh']
X_test_main = X_test.drop(['label', 'id_bh', 'id', 'job/role', 'from_date', 'to_date'], axis = 1)

In [ ]:
vec = DictVectorizer()
X_train = vec.fit_transform(X_train.to_dict('records')).toarray()
X_test_main = vec.fit_transform(X_test_main.to_dict('records')).toarray()

In [ ]:
X_train = np.concatenate((X_train,result[:27502]),axis=1)
X_test_main = np.concatenate((X_test_main,result[27502:]),axis=1)


In [ ]:
Y_train

0        5.0
1        4.0
2        2.0
3        5.0
4        6.0
        ... 
27497    6.0
27498    2.0
27499    5.0
27500    2.0
27501    2.0
Name: label, Length: 27502, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

multi_model = LGBMClassifier()
multi_model.fit(X_train, Y_train)

LGBMClassifier()

In [ ]:
prediction = multi_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(classification_report(Y_test,prediction))

              precision    recall  f1-score   support

         1.0       1.00      0.75      0.86        12
         2.0       0.94      0.93      0.93       988
         3.0       0.94      0.92      0.93       428
         4.0       0.92      0.95      0.93       775
         5.0       0.97      0.93      0.95       251
         6.0       0.91      0.91      0.91       294
         7.0       0.50      0.33      0.40         3

    accuracy                           0.93      2751
   macro avg       0.88      0.82      0.84      2751
weighted avg       0.93      0.93      0.93      2751



In [ ]:
prediction = multi_model.predict(X_test_main)

In [ ]:
guess = pd.DataFrame()
guess['id_bh'] = X_test_id
guess['label'] = list(map(int, prediction))
guess

,id_bh,label
27502,100000688,2
27503,100005503,2
27504,100006307,5
27505,100008625,6
27506,100009238,3
...,...,...
45631,9714758429,4
45632,9714758435,4
45633,9715056070,4
45634,9715637756,5


In [ ]:
guess.to_csv("prediction.csv", index=False)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/prediction-multimodal-timeseries.csv')
test = test.sort_values(['id_bh'])
test = test.reset_index(drop=True)
test = test['label']

print(classification_report(test,prediction))

              precision    recall  f1-score   support

           1       0.86      0.46      0.60        52
           2       0.93      0.93      0.93      6650
           3       0.94      0.89      0.91      2332
           4       0.90      0.95      0.92      5314
           5       0.91      0.94      0.92      1679
           6       0.90      0.84      0.87      2072
           7       1.00      0.31      0.48        35

    accuracy                           0.92     18134
   macro avg       0.92      0.76      0.81     18134
weighted avg       0.92      0.92      0.92     18134

